# Fake Structured Content Generation

Using a customized prompt, along with a few shot based prompting method to generate stuctured fake content for testing stuff out effectively

In [1]:
from getpass import getpass
GOOGLE_API_KEY = getpass()

 ········


In [2]:
from langchain_core.globals import set_llm_cache
from langchain_google_genai import GoogleGenerativeAI
from langchain_core.caches import InMemoryCache

llm = GoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=GOOGLE_API_KEY)

set_llm_cache(InMemoryCache())

In [3]:
raw_prompt = """You can an advanced fake data generator and you excel in generating and
filling out fake data in whatever structured format that is given to you.
Your job now is to generate fake data based on for the given json format.
You will get an input in this format number | input | schema
You need to generate [number] examples based on this [input] in this schema [schema]"""

In [4]:
json_schema = {
      "name": "patient name",
      "age": "patient age",
      "gender": "patient's inferred gender",
      "address": "patient address",
      "identity": "patient identity",
      "phone": "patient phone number",
      "problems": [
        {
          "name": "symptom name",
          "duration": "duration of symptom",
          "description": "description regarding the symptom"
        }
      ],
      "conditions": ["list of pre-existing conditions"],
      "description": "[AI GENERATED] Medically sounding description",
      "recommended_doctor": "[AI GENERATED] Suggested medical specialty"
    }

example = {
      "name": "Ishan",
      "age": "",
      "gender": "Male",
      "address": "Hyderabad",
      "identity": "",
      "phone": "",
      "problems": [
        {
          "name": "fever",
          "duration": "",
          "description": "High fever"
        },
        {
          "name": "headache",
          "duration": "",
          "description": "Headache localized in the forehead"
        },
        {
          "name": "cold",
          "duration": "",
          "description": "",
        }
      ],
      "conditions": ["diabetes"],
      "description": "[AI GENERATED] A male patient presenting with fever, headache, and cold symptoms, with a known diagnosis of diabetes.",
      "recommended_doctor": "[AI GENERATED] General Physician or Infectious Disease Specialist",
}

def make_prompt(number, example, schema):
    return f"{number} | {example} | {schema}"

import json
prompt = make_prompt(40, json.dumps(example), json.dumps(json_schema))

In [5]:
from google import genai
from google.genai import types

client = genai.Client(api_key=GOOGLE_API_KEY)
response = client.models.generate_content(
            model="gemini-2.0-flash",
            contents=[prompt],
            config=types.GenerateContentConfig(
                system_instruction=raw_prompt,
                temperature=1,
                response_mime_type="application/json",
            ),
        )

res = response.candidates[0].content.parts[0].text
print(res)

[
  {
    "name": "Aisha Khan",
    "age": "62",
    "gender": "Female",
    "address": "Mumbai",
    "identity": "Passport: P2947H8",
    "phone": "9876543210",
    "problems": [
      {
        "name": "Fatigue",
        "duration": "2 weeks",
        "description": "Persistent fatigue and weakness."
      },
      {
        "name": "Joint pain",
        "duration": "1 month",
        "description": "Pain in knees and elbows."
      },
      {
        "name": "Shortness of breath",
        "duration": "1 week",
        "description": "Difficulty breathing after mild exertion."
      }
    ],
    "conditions": [
      "Hypertension",
      "Osteoarthritis"
    ],
    "description": "[AI GENERATED] A 62-year-old female experiencing fatigue, joint pain, and shortness of breath, with a medical history of hypertension and osteoarthritis.",
    "recommended_doctor": "[AI GENERATED] Rheumatologist or General Practitioner"
  },
  {
    "name": "Rohan Verma",
    "age": "35",
    "gender": "M

In [6]:
obj = json.loads(res)
print(len(obj))

37


In [7]:
with open("patients.json", "w") as file:
    file.write(json.dumps({"patients": obj}))